In [ ]:
import numpy as np
import pandas as pd
import json

fine_path = 'data/train_fine.txt'
dic = set()
count = 1
l = 0
with open(fine_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        l = max(0, len(data['title']))
        for key in data['key_attr']:
            if key not in dic:
                dic.add(key)
                count += 1

print(key)
print(dic)


In [ ]:
print(l)

In [ ]:
import collections
import json
key_dic = {}
i = 0
with open('data/attr_to_attrvals.json', 'r', encoding='utf-8') as f:
    key_attr = json.load(f)
    for key in key_attr:
        key_dic[key] = i
        i += 1
print(key_dic)


In [ ]:
import os 
import numpy as np
import json
import random
import copy

data_path_1 = './data/train_fine.txt'
data_path_2 = './data/train_coarse.txt'

data_pos_list = []
data_neg_list = []

with open(data_path_1, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        if data['match']['图文'] == 1:
            data_pos_list.append(data)
        else:
            data_neg_list.append(data)

with open(data_path_2, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        if data['match']['图文'] == 1:
            data_pos_list.append(data)
        else:
            data_neg_list.append(data)

new_neg_list = []
for i in range(len(data_pos_list)):
    dic = copy.deepcopy(data_pos_list[i])
    index = random.randint(0, len(data_pos_list)-1)
    while index == i:
        index = random.randint(0, len(data_pos_list)-1)
    dic['title'] = data_pos_list[index]['title']
    dic['match']['图文'] = 0
    new_neg_list.append(dic)

print(len(data_pos_list), len(data_neg_list), len(new_neg_list))
all_data = data_pos_list + data_neg_list + new_neg_list
np.random.shuffle(all_data)
l = int(len(all_data) * 0.9)
x_train_list = all_data[:l]
x_val_list = all_data[l:]
# np.random.shuffle(x_val_list)

x_train_list = [json.dumps(dic, ensure_ascii=False)+'\n' for dic in x_train_list]
x_val_list = [json.dumps(dic, ensure_ascii=False)+'\n' for dic in x_val_list]


# with open('./data/pretrain_match.txt', 'w', encoding='utf-8') as f:
#     f.writelines(pre_ret)

with open('./data/new_train_match.txt', 'w', encoding='utf-8') as f:
    f.writelines(x_train_list)

with open('./data/new_val_match.txt', 'w', encoding='utf-8') as f:
    f.writelines(x_val_list)

# print(len(pre_ret))
print(len(x_train_list))
print(len(x_val_list))
print(len(x_train_list) + len(x_val_list))

In [ ]:
import json
new_dic = {}

with open('./data/attr_to_attrvals.json', 'r', encoding='utf-8') as f:
    attr_key = json.load(f)
    for key, value in attr_key.items():
        tmp = []
        for v in value:
            if '=' in v:
                tmp.append(v.split('='))
            else:
                tmp.append([v])
        new_dic[key] = tmp
print(new_dic)
rets = [json.dumps(new_dic, ensure_ascii=False)+'\n']
with open('./data/attr_match.json', 'w', encoding='utf-8') as f:
    f.writelines(rets)

In [2]:
import json
import numpy as np
from tqdm import tqdm

not_math_list = []
from random import choice

with open('./data/attr_match.json', 'r', encoding='utf-8') as f:
    attr_key = json.load(f)

def get_title_mask(title, key, val, attr_key):
    values = attr_key[key]
    key_index = 0
    for i in range(len(values)):
        if val in values[i]:
            key_index = i
            break
    new_index = np.random.randint(len(values))
    while new_index == key_index:
        new_index = np.random.randint(len(values))
    sub_val = values[new_index]
    new_sub_val = sub_val[np.random.randint(len(sub_val))]

    return title.replace(val, new_sub_val, 1)

def get_random_key(keys, ratio=0.7):
    list_ratio = [0.7, 0.9, 1]
    ratio = choice(list_ratio)
    l = int(len(keys) * ratio)
    if l == 0:
        l = 1
    np.random.shuffle(keys)
    return keys[:l]

count = 0
with open('./data/pos_coarse.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    new_dic = {}
    for line in tqdm(lines):
        data = json.loads(line)
        new_dic = data
        #print(new_dic['title'])
        keys = get_random_key([x for x in new_dic['match'].keys() if x != '图文'])
        for key in keys:
            new_dic['title'] = get_title_mask(new_dic['title'], key, new_dic['key_attr'][key], attr_key)
            new_dic['match'][key] = 0
        new_dic['match']['图文'] = 0
        #print(new_dic['title'])
        not_math_list.append(json.dumps(new_dic, ensure_ascii=False)+'\n')
        # print(new_dic['match'])
        # count += 1
        # print(count)
    print('YES')
print('yes')
print(len(not_math_list))       
with open('./data/neg_coarse.txt', 'w', encoding='utf-8') as f:
    f.writelines(not_math_list)

        

100%|██████████| 88679/88679 [02:45<00:00, 536.65it/s]


YES
yes
88679


In [ ]:
print(len(not_math_list))

In [ ]:
import json
import numpy as np
from tqdm import tqdm

not_math_list = []
from random import choice

with open('./data/attr_match.json', 'r', encoding='utf-8') as f:
    attr_key = json.load(f)

def get_title_mask(title, key, val, attr_key):
    values = attr_key[key]
    key_index = 0
    for i in range(len(values)):
        if val in values[i]:
            key_index = i
            break
    new_index = np.random.randint(len(values))
    while new_index == key_index:
        new_index = np.random.randint(len(values))
    sub_val = values[new_index]
    new_sub_val = sub_val[np.random.randint(len(sub_val))]

    return title.replace(val, new_sub_val, 1)

def get_random_key(keys, ratio=0.7):
    list_ratio = [0.7, 0.9, 1]
    ratio = choice(list_ratio)
    l = int(len(keys) * ratio)
    if l == 0:
        l = 1
    np.random.shuffle(keys)
    return keys[:l]

count = 0
with open('./data/pos_coarse.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    new_dic = {}
    for line in tqdm(lines):
        data = json.loads(line)
        new_dic = data
        keys = get_random_key([x for x in new_dic['match'].keys() if x != '图文'])
        for key in keys:
            new_dic['title'] = get_title_mask(new_dic['title'], key, new_dic['key_attr'][key], attr_key)
            new_dic['match'][key] = 0
        new_dic['match']['图文'] = 0
        not_math_list.append(json.dumps(new_dic, ensure_ascii=False)+'\n')
        
    print('YES')
print('yes')
print(len(not_math_list))       
with open('./data/neg_coarse.txt', 'w', encoding='utf-8') as f:
    f.writelines(not_math_list)

        

In [1]:
import json
with open('./data/attr_match.json', 'r', encoding='utf-8') as f:
    attr_key = json.load(f)
new_dic = {}
i = 0
for key, values in attr_key.items():
    for single_lis in values:
        for val in single_lis:
            new_dic[key+val] = i
            i += 1
print(new_dic)

{'领型高领': 0, '领型半高领': 1, '领型立领': 2, '领型连帽': 3, '领型可脱卸帽': 4, '领型翻领': 5, '领型衬衫领': 6, '领型POLO领': 7, '领型方领': 8, '领型娃娃领': 9, '领型荷叶领': 10, '领型双层领': 11, '领型西装领': 12, '领型U型领': 13, '领型一字领': 14, '领型围巾领': 15, '领型堆堆领': 16, '领型V领': 17, '领型棒球领': 18, '领型圆领': 19, '领型斜领': 20, '领型亨利领': 21, '袖长短袖': 22, '袖长五分袖': 23, '袖长九分袖': 24, '袖长长袖': 25, '袖长七分袖': 26, '袖长无袖': 27, '衣长超短款': 28, '衣长短款': 29, '衣长常规款': 30, '衣长长款': 31, '衣长超长款': 32, '衣长中长款': 33, '版型修身型': 34, '版型标准型': 35, '版型宽松型': 36, '裙长短裙': 37, '裙长超短裙': 38, '裙长中裙': 39, '裙长中长裙': 40, '裙长长裙': 41, '穿着方式套头': 42, '穿着方式开衫': 43, '类别手提包': 44, '类别单肩包': 45, '类别斜挎包': 46, '类别双肩包': 47, '裤型O型裤': 48, '裤型锥形裤': 49, '裤型哈伦裤': 50, '裤型灯笼裤': 51, '裤型铅笔裤': 52, '裤型直筒裤': 53, '裤型小脚裤': 54, '裤型工装裤': 55, '裤型紧身裤': 56, '裤型背带裤': 57, '裤型喇叭裤': 58, '裤型微喇裤': 59, '裤型阔腿裤': 60, '裤长短裤': 61, '裤长五分裤': 62, '裤长七分裤': 63, '裤长九分裤': 64, '裤长长裤': 65, '裤门襟松紧': 66, '裤门襟拉链': 67, '裤门襟系带': 68, '闭合方式松紧带': 69, '闭合方式拉链': 70, '闭合方式套筒': 71, '闭合方式套脚': 72, '闭合方式一脚蹬': 73, '闭合方式系带': 74, '闭合方式魔术贴': 75, '闭合方式搭扣': 76, '鞋帮高度高帮': 

In [2]:
with open('./data/attr_dic.json', 'w', encoding='utf-8') as f:
    json.dump(new_dic, f, ensure_ascii=False)

In [4]:
import numpy as np
from tqdm import tqdm

data_list = []
save_path = './data/train_5w_fine_attr.txt'

with open('./data/attr_match.json', 'r', encoding='utf-8') as f:
    attr_key = json.load(f)

def get_dismatch_value(key, val, attr_key):
    values = attr_key[key]
    key_index = 0
    for i in range(len(values)):
        if val in values[i]:
            key_index = i
            break
    new_index = np.random.randint(len(values))
    while new_index == key_index:
        new_index = np.random.randint(len(values))
    sub_val = values[new_index]
    new_sub_val = sub_val[np.random.randint(len(sub_val))]

    return new_sub_val

def get_all_attr(path):
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in tqdm(lines):
            data = json.loads(line)
            for key, value in data['key_attr'].items():
                new_dic = {}
                new_dic['feature'] = data['feature']
                new_dic['attr'] = key + value
                new_dic['attr_match'] = 1
                data_list.append(json.dumps(new_dic, ensure_ascii=False)+'\n')
                new_dic = {}
                new_dic['feature'] = data['feature']

                new_dic['attr'] = key + get_dismatch_value(key, value, attr_key)
                new_dic['attr_match'] = 0
                data_list.append(json.dumps(new_dic, ensure_ascii=False)+'\n')


attr_path_1 = './data/train_fine.txt'
get_all_attr(attr_path_1)
print(len(data_list))

with open(save_path, 'w', encoding='utf-8') as f:
    f.writelines(data_list)

100%|██████████| 50000/50000 [05:59<00:00, 139.25it/s]


219086


In [5]:
print(data_list[0])

{"feature": [1.1288161278, 0.0094874166, 0.0516808964, 0.496194005, 0.6149861217, -0.2549115419, -0.117655389, 0.8089491129, 0.0542874634, -1.143070817, -0.8707765341, 0.317706883, -0.2160565555, -0.5536744595, 0.0506066047, -0.1662157327, 0.3332067132, 0.500089705, -0.3963449299, 0.2887229025, -1.102519989, 0.4814974368, 0.3610215485, -0.8824773431, -0.068759352, 0.1462626159, -0.2247579396, -0.0337267257, -0.3110350668, -0.5177004337, 0.0085425405, 0.1234855652, -0.0096929222, -0.5021640658, -0.3868485093, 0.0132592572, 0.2814457715, 0.3890895545, -0.0168108791, 0.7196006179, 0.0108942389, 0.1500142962, -0.1680164039, -0.2112841308, -0.0882904828, -0.9946240187, -0.5320196748, 0.0796948671, 0.8765568733, -0.890999496, -1.4972048998, 0.1095142886, -0.0262572225, 0.6837017536, -0.4809237123, -0.1950486898, 0.0504611991, 0.0289665256, 0.060537003, 0.0103846854, 0.0135627426, -0.0754919052, 0.0135238003, 0.4657616019, -0.0073404759, -0.4350771308, -0.1683784127, 0.2758236825, -0.86862695